In [3]:
import pandas as pd
import pickle

In [4]:
# PART_2 was just an extended version of the first part, so ignore the first one
with open('capstone_1/beerFeeds/beerFeedDicts0to607_PART_2.pkl', 'rb') as f:
    feeds = pickle.load(f)
print(type(feeds), len(feeds))

<class 'dict'> 607


In [28]:
# to inspect the data
feed_gen = (k for k in feeds.keys())

In [15]:
feeds[next(feed_gen)].keys()

dict_keys(['newestID', 'oldestID', 'datalist'])

In [21]:
feeds[next(feed_gen)]['datalist'][0]['response']['checkins']['items'][0].keys()

dict_keys(['checkin_id', 'created_at', 'checkin_comment', 'rating_score', 'user', 'beer', 'brewery', 'venue', 'comments', 'toasts', 'media', 'source', 'badges'])

In [56]:
to_concat = [pd.io.json.json_normalize(data['response']['checkins']['items'])
             for k in feeds.keys() for data in feeds[k]['datalist']]

In [57]:
len(to_concat)

4690

In [61]:
# inspect
c = to_concat[0].columns

# here were the checkin feed cols
keepcols = ['checkin_id', 'beer.bid', 'user.uid', 'rating_score', 'beer.beer_abv', 
          'brewery.brewery_name','beer.beer_name', 'beer.beer_style',  
           'brewery.brewery_id', 'brewery.brewery_type', 'brewery.country_name',
          'brewery.location.brewery_city', 'brewery.location.brewery_state',
          'brewery.location.lat', 'brewery.location.lng', 'user.user_name',
          'venue.categories.items', 'venue.location.lat', 'venue.location.lng',
         'venue.location.venue_city', 'venue.location.venue_country', 
          'venue.location.venue_state', 'venue.primary_category',
          'venue.venue_id', 'checkin_comment','created_at', 
       ]
[col for col in keepcols if col not in c]

['brewery.brewery_type']

In [37]:
# add in some useful ones from beer feeds:
keepcols = keepcols[:7] + ['beer.beer_description', 'beer.beer_ibu'] + keepcols[7:9] + keepcols[10:]

In [58]:
beer_cat = pd.concat(to_concat, axis=0, ignore_index=True)
# slim the frame
beer_cat = beer_cat[keepcols]
beer_cat.shape

(114121, 27)

In [45]:
beer_cat.head()

,checkin_id,beer.bid,user.uid,rating_score,beer.beer_abv,brewery.brewery_name,beer.beer_name,beer.beer_description,beer.beer_ibu,beer.beer_style,...,venue.categories.items,venue.location.lat,venue.location.lng,venue.location.venue_city,venue.location.venue_country,venue.location.venue_state,venue.primary_category,venue.venue_id,checkin_comment,created_at
0,819020801,1796100,288533,4.00,6.7,Hi-Wire Brewing,Hi-Pitch Mosaic IPA,A balanced Western North Carolina IPA with bri...,55,IPA - American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,"Sat, 19 Oct 2019 11:33:05 +0000"
1,818960695,1796100,643191,4.00,6.7,Hi-Wire Brewing,Hi-Pitch Mosaic IPA,A balanced Western North Carolina IPA with bri...,55,IPA - American,...,"[{'category_key': 'brewery', 'category_name': ...",33.9715,-81.0281,Columbia,United States,SC,Nightlife Spot,1859021.0,,"Sat, 19 Oct 2019 03:37:37 +0000"
2,818960128,1796100,3497038,3.00,6.7,Hi-Wire Brewing,Hi-Pitch Mosaic IPA,A balanced Western North Carolina IPA with bri...,55,IPA - American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,"Sat, 19 Oct 2019 03:36:06 +0000"
3,818957618,1796100,4006238,3.75,6.7,Hi-Wire Brewing,Hi-Pitch Mosaic IPA,A balanced Western North Carolina IPA with bri...,55,IPA - American,...,"[{'category_key': 'lounge', 'category_name': '...",35.9479,-86.8100,Franklin,United States,TN,Nightlife Spot,9496119.0,,"Sat, 19 Oct 2019 03:29:15 +0000"
4,818956577,1796100,3132998,0.00,6.7,Hi-Wire Brewing,Hi-Pitch Mosaic IPA,A balanced Western North Carolina IPA with bri...,55,IPA - American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,"Sat, 19 Oct 2019 03:26:22 +0000"


In [64]:
beer_cat.to_csv('capstone_1/beercat.csv')